In [1]:
# COMEX Gold Inventory Analysis & Plotting (Plotly)
# ------------------------------------------------
# Differences vs Silver/Copper:
# - Unit: Troy Ounces
# - Categories: Registered / Pledged / Eligible / Total
# - Pledged gold matters (collateralized, not freely deliverable)

import os
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# =========================
# 1. Path & file pattern
# =========================

DATA_DIR = r"E:\code\python\mytools\SystemMacro\CME\CME_Stocks\Gold"
FILE_PATTERN = re.compile(r"Gold_Stocks_(\d{8})\.xls")

# =========================
# 2. Read single CME Gold file
# =========================

def read_single_file(file_path, date_str):
    df = pd.read_excel(file_path, header=None)

    records = []
    current_depository = None

    INVALID_KEYWORDS = ["TOTAL", "GOLD", "ENHANCED"]

    for _, row in df.iterrows():
        cell = str(row[0]).strip()

        # Depository name
        if (
            cell.isupper()
            and "DEPOSITORY" not in cell
            and not any(k in cell for k in INVALID_KEYWORDS)
        ):
            current_depository = cell
            continue

        # Category rows
        if cell in ["Registered", "Eligible", "Pledged", "Total"] and current_depository:
            try:
                ounces = float(row.dropna().iloc[-1])
            except Exception:
                continue

            records.append({
                "Date": pd.to_datetime(date_str),
                "Depository": current_depository,
                "Category": cell,
                "Ounces": ounces
            })

    return pd.DataFrame(records)

# =========================
# 3. Load all files
# =========================

all_dfs = []

for fname in os.listdir(DATA_DIR):
    match = FILE_PATTERN.match(fname)
    if match:
        date_str = match.group(1)
        fpath = os.path.join(DATA_DIR, fname)
        print(f"Reading {fname}")
        all_dfs.append(read_single_file(fpath, date_str))

df_all = pd.concat(all_dfs, ignore_index=True)

# =========================
# 4. Market-level aggregation
# =========================

market_df = (
    df_all
    .groupby(["Date", "Category"], as_index=False)["Ounces"]
    .sum()
)

pivot_market = (
    market_df
    .pivot(index="Date", columns="Category", values="Ounces")
    .sort_index()
)

pivot_market["Free_Float"] = pivot_market["Registered"] + pivot_market["Eligible"]
pivot_market["Registered_Ratio"] = pivot_market["Registered"] / pivot_market["Free_Float"]

pivot_market["dRegistered"] = pivot_market["Registered"].diff()
pivot_market["dEligible"] = pivot_market["Eligible"].diff()
pivot_market["dPledged"] = pivot_market.get("Pledged", 0).diff()
pivot_market["dTotal"] = pivot_market["Total"].diff()

# =========================
# 5. Plot 1: Inventory levels (multi Y-axis)
# =========================

fig_inventory = go.Figure()

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Registered"],
    name="Registered",
    yaxis="y1"
))

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Eligible"],
    name="Eligible",
    yaxis="y2"
))

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Pledged"],
    name="Pledged",
    yaxis="y2"
))

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Total"],
    name="Total",
    yaxis="y3"
))

fig_inventory.update_layout(
    title="COMEX Gold Inventory (Multi Y-Axis)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Registered (oz)"),
    yaxis2=dict(overlaying="y", side="right", title="Eligible / Pledged"),
    yaxis3=dict(overlaying="y", side="right", position=0.95, title="Total"),
)

fig_inventory.show()

# =========================
# 6. Plot 2: Registered / Free Float ratio
# =========================

fig_ratio = px.line(
    pivot_market,
    y="Registered_Ratio",
    title="Registered / (Registered + Eligible)"
)
fig_ratio.show()

# =========================
# 7. Plot 3: Daily net change
# =========================

fig_flow = go.Figure()
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dRegistered"], name="Δ Registered")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dEligible"], name="Δ Eligible")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dPledged"], name="Δ Pledged")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dTotal"], name="Δ Total")

fig_flow.update_layout(
    barmode="group",
    title="Daily Inventory Net Change (Gold, oz)"
)
fig_flow.show()

# =========================
# 8. Plot 4: Registered by depository (stacked area)
# =========================

reg_df = df_all[df_all["Category"] == "Registered"]

reg_df = (
    reg_df
    .groupby(["Date", "Depository"], as_index=False)["Ounces"]
    .sum()
)

fig_stack = px.area(
    reg_df,
    x="Date",
    y="Ounces",
    color="Depository",
    title="Registered Gold by Depository"
)
fig_stack.show()

# =========================
# 9. Plot 5: ΔRegistered heatmap
# =========================

reg_pivot = (
    reg_df
    .pivot(index="Date", columns="Depository", values="Ounces")
    .sort_index()
)

reg_diff = reg_pivot.diff()

fig_heat = px.imshow(
    reg_diff.T,
    aspect="auto",
    color_continuous_scale="RdBu",
    title="Δ Registered Gold by Depository"
)
fig_heat.show()

# =========================
# 10. Print key tables
# =========================

print("\n=== Market-level Gold Inventory (last 10 days) ===")
print(
    pivot_market[["Registered", "Eligible", "Pledged", "Total", "Registered_Ratio"]]
    .tail(10)
    .round(4)
)

print("\n=== Daily Net Changes (last 10 days) ===")
print(
    pivot_market[["dRegistered", "dEligible", "dPledged", "dTotal"]]
    .tail(10)
)

latest_date = reg_df["Date"].max()
print(f"\n=== Latest Registered by Depository ({latest_date.date()}) ===")
print(
    reg_df[reg_df["Date"] == latest_date]
    .sort_values("Ounces", ascending=False)
    .reset_index(drop=True)
)


Reading Gold_Stocks_20251204.xls
Reading Gold_Stocks_20251205.xls
Reading Gold_Stocks_20251208.xls
Reading Gold_Stocks_20251209.xls
Reading Gold_Stocks_20251210.xls
Reading Gold_Stocks_20251211.xls
Reading Gold_Stocks_20251212.xls
Reading Gold_Stocks_20251215.xls
Reading Gold_Stocks_20251216.xls
Reading Gold_Stocks_20251217.xls
Reading Gold_Stocks_20251218.xls
Reading Gold_Stocks_20251219.xls
Reading Gold_Stocks_20251222.xls
Reading Gold_Stocks_20251223.xls



=== Market-level Gold Inventory (last 10 days) ===
Category      Registered      Eligible      Pledged         Total  \
Date                                                                
2025-12-10  1.880227e+07  1.730735e+07  2010707.547  3.610962e+07   
2025-12-11  1.893651e+07  1.717910e+07  2010900.453  3.611561e+07   
2025-12-12  1.895362e+07  1.701413e+07  1998128.729  3.596775e+07   
2025-12-15  1.905839e+07  1.690884e+07  1991242.509  3.596723e+07   
2025-12-16  1.919188e+07  1.679946e+07  1991242.509  3.599134e+07   
2025-12-17  1.922468e+07  1.676667e+07  1980647.438  3.599134e+07   
2025-12-18  1.927609e+07  1.679407e+07  1969701.020  3.607016e+07   
2025-12-19  1.929856e+07  1.670736e+07  2015413.233  3.600592e+07   
2025-12-22  1.920674e+07  1.691335e+07  2015413.233  3.612009e+07   
2025-12-23  1.932190e+07  1.683736e+07  2001427.554  3.615927e+07   

Category    Registered_Ratio  
Date                          
2025-12-10            0.5207  
2025-12-11            0.52